# Arytmetyka w ciałach Galois

## Co to jest ciało Galois?

**Ciałem Galois** nazywamy ciało $(G,+,\cdot)$ o skończonej liczbie elementów. Najprostszym przykładem takiego ciała są $\mathbb{Z}_p$, gdzie $p$ jest liczbą pierwszą. *Rzędem* ciała skończonego nazywamy liczbę jego elementów. Ciała skończone tego samego rzędu są izomorficzne.

Kolejnym ważnym przykładem ciała Galois jest pierścień ilorazowy $\mathbb{Z}_p[X]/W(X)$, gdzie $p$ jest liczbą pierwszą a $W$ jest nierozkładalnym wielomianem monicznym stopnia $n$.

## Pierścień $\mathbb{Z}_n$

W ciele $\mathbb{Z}$ wprowadzamy relację równoważności $\mod n$ (gdzie $n$ jest ustaloną dodatnią liczbą naturalną):
$$a\equiv_n b\Leftrightarrow [a]_n=[b]_n$$
gdzie $[]_n$ oznacza resztę z dzielenia przez $n$.

Arytmetyka $\mod n$:$$a+b=[a+b]_n$$ $$ab=[ab]_n$$

**Pierwiastkiem pierwotnym** $\mod n$ nazywamy liczbę, której potęgi $\mod n$ dają wszystkie reszty z dzielenia przez $n$, które są względnie pierwsze z $n$. Pierwiastek pierwotny istnieje tylko dla następujących $n$:
- $n=p^k$, gdzie $p$ jest liczbą pierwszą różną od 2
- $n=2p^k$, gdzie $p$ - j.w.
- $n=2$ lub $n=4$

### Sage math:
Konstruujemy pierścień `R=Integers(n)` lub `R=IntegerModRing(n)`, gdzie za `n` podajemy ustaloną liczbę naturalną. Jeżeli chcemy poznać postać liczby `x` w tym pierścieniu, to piszemy `R(x)`. Inną opcją jest funkcja `mod(x,n)`

In [23]:
R=Integers(3)
x=R(4)
print(x)
print(type(x))

NameError: name 'Integers' is not defined

In [ ]:
R(2+7)

In [ ]:
R(2*4)

In [ ]:
RR=IntegerModRing(5)
x=RR(10)
print(x)
print(type(x))

In [ ]:
x=mod(10,4)
print(x)
print(type(x))

Pierwiastki pierwotne w Sage znajdujemy funkcją `primitive_root(n)`.

In [ ]:
x=primitive_root(7)
print(x)
print(type(x))

In [ ]:
primitive_root(5)

### python:

W pythonie nie poszalejemy - operator `%` zwraca resztę z dzielenia. I to tyle. Funkcje do arytmetyki mod $n$ można znaleźć w module SymPy.

In [ ]:
x=5%2
print(x)
print(type(x))

## Zadanie 1.
Zaimplementować w pythonie funkcję `prim_root(n)` znajdującą pierwiastki pierwotne mod $n$. Jeżeli taki pierwiastek nie istnieje funkcja ma zwrócić 0. Jeżeli takich pierwiastków jest więcej funkcja ma zwrócić najmniejszy z nich.

In [194]:
def prim_root(modulo):
    coprime_set = {num for num in range(1, modulo) if gcd(num, modulo) == 1}
    roots = []
    for g in range(1, modulo):
        powers_set = {pow(g, powers, modulo) for powers in range(1, modulo)}
        if coprime_set == powers_set:
            roots.append(g)
    
    if roots == []:
        return 0
    roots.sort()
    return roots[0]




In [195]:
#TESTY
try:
    assert prim_root(7)==3
    assert prim_root(15)==0
    assert prim_root(26)==7
except:
    print('Próbuj dalej')
    raise

## Zadanie 2.

Zaimplementować w Pythonie klasę `Z7()`, której obiekty to pierścienie reszt z dzielenia przez 7. Przeładować operatory `+`, `-`, `*`, aby na obiektach klasy wykonywały działania mod 7 oraz metodę `__repr__`.

In [80]:
class Z7():
    
    def __init__(self,a):
        self.a = a % 7

    def __radd__(self,x):
        if isinstance(x, int):
            x = Z7(x)
        return Z7(self.a + x.a)
    
    def __add__(self,x):
        if isinstance(x, int):
            x = Z7(x)
        return Z7(self.a + x.a)

    def __sub__(self,x):
        if isinstance(x, int):
            x = Z7(x)
        return Z7(self.a - x.a)

    def __mul__(self,x):
        if isinstance(x, int):
            x = Z7(x)
        return Z7(self.a * x.a)

    def __pow__(self, x):
        if isinstance(x,int):
            x = Z7(x)
        return Z7(pow(self.a, x.a,7))

    def __repr__(self) -> str:
        return str(self.a)
        
    
    
        


In [81]:
#TESTY

x=Z7(2)
y=Z7(10)
z=Z7(14)
print(x,y,z)
#2 3 0
print(x+z, y*x,z+x)
#2 6
print(x + 2 , 2 + x)


2 3 0
2 6 2
4 4


### Wymiana klucza typu Diffie-Hellman z wykorzystaniem pierwiastka pierwotnego

Alice i Bob uzgadniają klucz publiczny będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in (0,p)$
- sekret Boba: liczba całkowita $m\in (0,p)$
- Alice generuje $x=q^n$ i wysyła do Boba
- Bob generuje $y=q^m$ i wysyła Alice
- Alice oblicza klucz $k=y^n$
- Bob oblicza klucz $k=x^m$

Użycie pierwiastka pierwotnego gwarantuje, że wielkość klucza $k$ nie przekroczy $p-1$.

## Zadanie 3.

Zaimplementuj w Sage lub pythonie powyższy prosty algorytm wymiany klucza. Przyda się funkcja `random_prime()` (Sage) oraz `randint()`.

In [164]:
from random import randint


class Diffe_Hellman:
    def __init__(self,p,q) -> None:
        self.p = p
        self.q = q
        self.n = randint(1,p)
        self.x = pow(q,self.n,p)
        self.k = None
    
    def exchange(self) -> int:
        return self.x
    
    def decode(self, other_side) -> int:
        y = other_side.exchange()
        self.k = pow(y, self.n, p)
        return self.k
    
p = 31
q = 3
    
Alice = Diffe_Hellman(p,q)
Bob = Diffe_Hellman(p,q)
print(Alice.n,Bob.n)
print(Alice.decode(Bob), Bob.decode(Alice))


1 2
9 9


## Pierścienie ilorazowe wielomianów

Aby utworzyć pierścień ilorazowy $\mathbb{Z}_n[X]/W(X)$ w Sage musimy najpierw utworzyć $\mathbb{Z}_n[X]$, czyli pierścień wielomianów o współczynnikach z $\mathbb{Z}_n$:

`R=PolynomialRing(Integers(n),'X')`

Jeżeli w dalszej części kodu mamy zamiar korzystać z wielomianów z tego pierścienia, to dobrze jest rozdzielić nazewnictwo zmiennych niezależnych

`X=R.gen()`

Teraz każdy wielomian zmiennej `X` będzie przez Sage traktowany jako element pierścienia `R`.

In [92]:
R=PolynomialRing(Integers(5),'X')
X=R.gen()

X^6-13*X^4+12*X^2-10*X+6

NameError: name 'PolynomialRing' is not defined

Pierścień ilorazowy tworzymy metodą `R.quotient(W,'x')`, gdzie `W` jest dowolnym wielomianem. Podobnie jak poprzednio dobrze jest od razu zdefiniować `x` jako zmienną niezależną wielomianów z nowego pierścienia.

In [ ]:
Rq=R.quotient(X^4+1,'x')
x=Rq.gen()

x^6-13*x^4+12*x^2-10*x+6

In [ ]:
w1=7*x^6+14
w2=24*x^4-5*x^2-7*x+13

expand(w1*w2)

## Zadanie 4.

Zaimplementować w pythonie arytmetykę pierścienia ilorazowego wielomianów utożsamiając wielomian z wektorem współczynników przy poszczególnych potęgach.